In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [2]:
batch_size = 64
num_epochs = 100
extract_size = 2048
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
encoder = Encoder()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train.h')

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [8]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = FinalClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [9]:
classify_model.load_state_dict(torch.load('./models/class_single_model_last.pkl'))

### time to train model

In [12]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.001)

In [13]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(401, num_epochs+401):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_model_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_model_last.pkl'))

Epoch [401/100], Step [500/511], Loss: 0.6172, Accuracy: 84.38%, Best_acc: 90.62%            
 Epoch 401, spent time:6.37s
Epoch [402/100], Step [500/511], Loss: 0.8393, Accuracy: 84.38%, Best_acc: 93.75%            
 Epoch 402, spent time:6.53s
Epoch [403/100], Step [500/511], Loss: 0.5647, Accuracy: 90.62%, Best_acc: 93.75%            
 Epoch 403, spent time:6.44s
Epoch [404/100], Step [500/511], Loss: 0.7312, Accuracy: 85.94%, Best_acc: 93.75%            
 Epoch 404, spent time:6.44s
Epoch [405/100], Step [500/511], Loss: 0.7550, Accuracy: 84.38%, Best_acc: 93.75%            
 Epoch 405, spent time:6.40s
Epoch [406/100], Step [500/511], Loss: 0.6186, Accuracy: 89.06%, Best_acc: 93.75%            
 Epoch 406, spent time:6.43s
Epoch [407/100], Step [500/511], Loss: 0.7193, Accuracy: 84.38%, Best_acc: 93.75%            
 Epoch 407, spent time:6.40s
Epoch [408/100], Step [500/511], Loss: 0.7707, Accuracy: 85.94%, Best_acc: 95.31%            
 Epoch 408, spent time:6.43s
Epoch [409/100],

Epoch [467/100], Step [500/511], Loss: 0.8885, Accuracy: 85.94%, Best_acc: 96.88%            
 Epoch 467, spent time:6.52s
Epoch [468/100], Step [500/511], Loss: 0.7439, Accuracy: 81.25%, Best_acc: 96.88%            
 Epoch 468, spent time:6.49s
Epoch [469/100], Step [500/511], Loss: 0.7394, Accuracy: 82.81%, Best_acc: 96.88%            
 Epoch 469, spent time:6.41s
Epoch [470/100], Step [500/511], Loss: 0.7116, Accuracy: 81.25%, Best_acc: 96.88%            
 Epoch 470, spent time:6.43s
Epoch [471/100], Step [500/511], Loss: 0.9060, Accuracy: 82.81%, Best_acc: 96.88%            
 Epoch 471, spent time:6.42s
Epoch [472/100], Step [500/511], Loss: 0.7595, Accuracy: 87.50%, Best_acc: 96.88%            
 Epoch 472, spent time:6.45s
Epoch [473/100], Step [500/511], Loss: 0.7339, Accuracy: 90.62%, Best_acc: 96.88%            
 Epoch 473, spent time:6.47s
Epoch [474/100], Step [500/511], Loss: 0.5868, Accuracy: 89.06%, Best_acc: 96.88%            
 Epoch 474, spent time:6.47s
Epoch [475/100],

### How good is the model?

In [6]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_new.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid.h")

In [29]:
classify_model.load_state_dict(torch.load('./models/class_single_model_last.pkl'))

In [30]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 62.42%.
